# Fractional Cascading

## Problem Statement

Given a `k` list of sorted integers and a value `x`. Given a query value `x` return the largest value less than or equal to `x` in each of the `k` lists.

In [1]:
import bisect

In [2]:
arr = [
    [1, 4, 6, 7, 10, 11],
    [0, 2, 3, 4, 5, 20],
    [4, 6, 10, 11, 15, 26],
    [2, 3, 7, 17, 23, 29],
]

x = 4

## Slower Queries Less Space | Naive Way - `k` binary searches

In [3]:
def binary_search(nums, q): 
    return bisect.bisect_left(nums, q)

[binary_search(l, x) for l in arr]

[1, 3, 0, 2]

## Faster Queries More Space | Global Ordering

In [4]:
# For each of the element in `arr` pointers will hold
# the location where the element `arr[i][j]` will be inserted.
import numpy
pointers =  numpy.zeros((len(arr), len(arr[0]), len(arr)), dtype=int)

In [5]:
for i, l in enumerate(arr):
    for j, e in enumerate(l):
        for k, m in enumerate(arr):
            pointers[i][j][k] = int(bisect.bisect_left(m, e))

See where `17` - 4th list 4th element

In [6]:
pointers[3][3]

array([6, 5, 5, 3])

In [7]:
ptr_arr = sorted([
    (y, pointers[i][j],)
    for i, x in enumerate(arr)
        for j, y in enumerate(x)
], key=lambda x: x[0])

In [8]:
ptr_arr[:5]

[(0, array([0, 0, 0, 0])),
 (1, array([0, 1, 0, 0])),
 (2, array([1, 1, 0, 0])),
 (2, array([1, 1, 0, 0])),
 (3, array([1, 2, 0, 1]))]

In [9]:
U = list(zip(*ptr_arr))[0]

In [10]:
list(ptr_arr[bisect.bisect_left(U, x)][1])

[1, 3, 0, 2]

## Best of both Worlds | Fractional Cascading